In [1]:
import numpy as np
import pandas as pd
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.preprocessing import MinMaxScaler,StandardScaler,RobustScaler
from sklearn.metrics import accuracy_score,roc_auc_score,average_precision_score,f1_score,roc_curve,auc
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')
from fastai.tabular.all import *
from collections import Counter
import itertools
import os
from imblearn.over_sampling import SMOTE,ADASYN,SVMSMOTE
from statistics import mean 
import warnings
warnings.filterwarnings('ignore')
from imblearn.combine import SMOTETomek,SMOTEENN
from matplotlib import pyplot as plt


In [3]:
p = "TMGWO"
x = "NASA/" + p
loc_fs_df = "C:/Users/shubh/Desktop/XGB DNN/df after fs/AEEEM"
loc_orig = "C:/Users/shubh/Desktop/XGB DNN/Dataset/AEEEM_corred_duplicate-Copy"

In [7]:
skf = StratifiedKFold(n_splits=10)
sampler = SMOTETomek(n_jobs=-1,random_state=0)
scaler = MinMaxScaler()

dx = pd.DataFrame()
fname = []
l=[]
m=[]
n=[]
dy= pd.DataFrame()
for file in os.listdir(loc_fs_df):
     #if file.endswith('_16.csv'):
        dr = pd.DataFrame()
    
        fname.append(file.split('.csv')[0])
        df_orig = pd.read_csv(os.path.join(loc_fs_df,file))
        df_orig = df_orig.iloc[:,1:]
        X_orig = df_orig.iloc[:,:-1]
        y_orig = df_orig.iloc[:,-1]

        a=[]
        b=[]
        c=[]

        e=[]
        k = 0
        for train_index, test_index in skf.split(X_orig, y_orig):

            xtrain, xtest = X_orig.iloc[train_index], X_orig.iloc[test_index]
            ytrain, ytest = y_orig.iloc[train_index], y_orig.iloc[test_index]    
            

            cont_names = X_orig.columns.tolist()
            #cont_names.remove('Defective')
            
#             X_train_sm,y_train_sm = sampler.fit_resample(xtrain,ytrain)
            
#             i  = scaler.fit_transform(X_train_sm)
#             j  = scaler.transform(xtest)
#             X_train_sm_sc = pd.DataFrame(i,columns=X_orig.columns)
#             X_test_sm_sc  = pd.DataFrame(j,columns=X_orig.columns)
           
            df_train = pd.concat([xtrain,ytrain],axis=1)
            #df_train.columns = df_orig.columns.tolist()
            df_test = pd.concat([xtest,ytest],axis=1)
            #df_test.columns = df_orig.columns.tolist()

#             X = df_train.iloc[:,:-1]
#             y = df_train.iloc[:,-1]

#             X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=0,stratify=y)

#             df_train1 = pd.concat([X_train,y_train],axis = 1)

#             X_train_sm,y_train_sm = sampler.fit_resample(X_train,y_train)

#             df_train_sm = pd.concat([X_train_sm,y_train_sm],axis = 1)
            

            b1 = int(xtrain.shape[0]/4)

            dls = TabularDataLoaders.from_df(df_train, y_names="Defective",bs=64,
                                            cont_names = cont_names,
                                            procs = [Categorify, FillMissing, Normalize],
                                            y_block = CategoryBlock())
            set_seed(0)
            learner = tabular_learner(dls, layers=[512,256],
                                      metrics=accuracy,
                                      loss_func=BCEWithLogitsLoss()
                              
                                      
                            
                                      
                                      
                                      
                                      
                                      
                                     )                                      

            lr_min, lr_steep, lr_valley, lr_slide = learner.lr_find(suggest_funcs=(minimum, steep, valley, slide))
           
            fig = 'C:/Users/shubh/Desktop/XGB DNN/Results/NASA/DNN//' + str(file.split(".csv")[0]) + "_" + str(k) + ".jpg"
            plt.savefig(fig)
            k=k+1

            learner.fit_one_cycle(1000,lr_max = lr_valley,
                                  cbs=[EarlyStoppingCallback(monitor='valid_loss', min_delta=0.001, patience=50),
                                    SaveModelCallback(monitor='valid_loss', min_delta=0.001)])


            test_dl = learner.dls.test_dl(df_test)
            preds,y1 = learner.get_preds(dl=test_dl)
            probs = np.exp(preds[:,1])
            acc = accuracy(preds,y1)
            fpr, tpr, thresholds = roc_curve(y1, probs, pos_label=1)
            roc = auc(fpr, tpr)
            prc = average_precision_score(y1,probs)


            a.append(acc.item())
            b.append(roc)
            c.append(prc)
#         a = [m for m in a if m in sorted(a)[-9:]]
#         b = [m for m in b if m in sorted(b)[-9:]]
#         c = [m for m in c if m in sorted(c)[-9:]]
        #for i in [mean(a),mean(b),mean(c)]:
            #e.append(i)
        dr = pd.concat([dr,pd.Series(a),pd.Series(b),pd.Series(c)],axis=1)
        dr.columns = ['Accuracy','ROC','AUPRC']
        dr.index = ['Fold1','Fold2','Fold3','Fold4',
                       'Fold5','Fold6','Fold7','Fold8','Fold9','Fold10']
        loc_results = 'C:/Users/shubh/Desktop/XGB DNN/Results/NASA/DNN/' + str(file)
        dr.to_csv(loc_results)
        dx = pd.concat([dx,pd.Series(min(a)),pd.Series(mean(a)),pd.Series(max(a)),
                   pd.Series(min(b)),pd.Series(mean(b)),pd.Series(max(b)),
                   pd.Series(min(c)),pd.Series(mean(c)),pd.Series(max(c))])
        l.append(mean(a))
        m.append(mean(b))
        n.append(mean(c))
dx.to_csv('C:/Users/shubh/Desktop/XGB DNN/Results/NASA//dnn.csv')
    
dy = pd.concat([dy,pd.Series(l),pd.Series(m),pd.Series(n)],axis=1)
dy.columns = ['Accuracy','ROC','AUPRC']
dy.index = fname
dy.to_csv('C:/Users/shubh/Desktop/XGB DNN/Results/NASA//dnn_mean.csv')
    

NameError: name 'BCEWithLogitsLoss' is not defined

In [5]:
dy.to_csv('C:/Users/shubh/Desktop/XGB DNN/Results/NASA/DNN/fs/dnn_mean.csv')
b

[0.8718316594930655,
 0.9439890710382514,
 0.9002732240437159,
 0.942167577413479,
 0.9025500910746812,
 0.8656648451730419,
 0.8991347905282332,
 0.9635701275045537,
 0.8897996357012751,
 0.9238187078109932]

In [10]:
dy.columns = ['Accuracy','ROC','AUPRC']
dy.index = fname
dy.to_csv('C:/Users/shubh/Desktop/XGB DNN/Results/NASA/DNN/dnn_mean.csv')

In [4]:
dr

,Accuracy,ROC,AUPRC
Fold1,0.864286,0.799617,0.405206
Fold2,0.900000,0.935792,0.703458
Fold3,0.914286,0.899818,0.665703
Fold4,0.900000,0.925774,0.689916
Fold5,0.914286,0.929872,0.754040
Fold6,0.907143,0.916667,0.697148
Fold7,0.864286,0.878643,0.612493
Fold8,0.928571,0.963570,0.835161
Fold9,0.892857,0.871129,0.656806
Fold10,0.892086,0.939730,0.620880


In [5]:
dx.to_csv("1.csv")

In [6]:
fname.to

['cm1', 'jm1', 'kc1', 'kc3', 'mc1', 'mc2', 'mw1', 'pc1', 'pc2', 'pc3', 'pc4']

In [ ]:
file